#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO  
-CIMENTRO VOTORAN  
-Magalu   

#Aplicador formula de taxas  
#Inserir no template dos marketplaces
  

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


In [ ]:
#Baixa os anexos XML dos emails

save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [ ]:
# Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    data_str = root.find(".//ns:ide/ns:dhEmi", ns).text
    data_emissao = datetime.fromisoformat(data_str)

    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        temp_codigo = det.find("./ns:prod/ns:cProd", ns).text
        if not "-" in temp_codigo:
            produto ['Codigo Produto'] = temp_codigo
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produto ['Data Emissão'] = data_emissao
        produtos.append(produto)
    return produtos

caminho = 'notas/35200213571969000164550010013235011005754650-nfe-env.xml'    
produtos = extrai_dados(caminho)
todos_produtos.extend(produtos) 

# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns)
            pasta_destino = os.path.join("notas/")

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")



fornecedores_pesados = [
    'IND E COM DE TUBOS E CONEXOES FORT.COM',
    'VOTORANTIM CIMENTOS SA',
    'CABOQUINHO MATERIAIS PARA CONSTRUCAO'
]


produtos = pd.DataFrame(todos_produtos)
produtos = produtos[~produtos['Fornecedor'].isin(fornecedores_pesados)]
produtos = produtos.sort_values(by="Data Emissão", ascending=False)

produtos = produtos.drop_duplicates(subset='Codigo Produto', keep='first')



In [ ]:
produtos['Fornecedor'].unique()



In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
}



In [ ]:
for  index, produto in produtos.iterrows():
    fornecedor = produto['Fornecedor']
    codigo_produto = produto['Codigo Produto']
    descricao = produto['Descrição']
    try:
        if fornecedor == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':   
            url = f'https://www.construdigi.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == 'M.S.B. COMERCIO DE MATERIAIS PARA CONSTRUCAO':
            url = f'https://msbitaqua.com.br/produto/{codigo_produto}/{descricao}'
        elif fornecedor == "CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU":
            url = f'https://www.construja.com.br/produto/{codigo_produto}/{descricao}'
        else:
            print('Indisponivel')
            continue
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        script = soup.find("script", type="application/json")
        data = json.loads(script.string) if script else {}

        extrai = data.get("props", {}).get("pageProps", {}).get("produto", {})
        url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
        marca = next((p.get("desc") for p in extrai.get("dimensoes", []) if p.get("label") == "MARCA"), "Não disponível")
        peso = extrai.get("pesoBruto", "Não disponível")
        if codigo_barras == 'SEM GTIN':
            codigo_barras = extrai.get("codBarra", "SEM GTIN")
        produtos.at[index, 'Código de Barras'] = codigo_barras
        produtos.at[index, 'Peso'] = peso
        produtos.at[index, "Marca"] = marca
        produtos.at[index, 'Url Imagem'] = url_img
        print(produto['Descrição'])
        
    except Exception as e:
        print(e)


produtos.to_excel('produtos.xlsx', index=False)


In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright




async def run():

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))

        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()




In [ ]:
import requests
from bs4 import BeautifulSoup
import json

response = requests.get('https://msbitaqua.com.br/produto/11748/11748')
soup = BeautifulSoup(response.text, "html.parser")
script = soup.find("script", type="application/json")
data = json.loads(script.string)

produto = data['props']['pageProps']['produto']
produto

In [ ]:
import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://msbitaqua.com.br/produto/2123/2123'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            print(data.keys())
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from env import pwd, user
from datetime import date, datetime
import requests
from bs4 import BeautifulSoup
import json
import time


import json
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def run():

    async with async_playwright() as p:
        url = 'https://www.construdigi.com.br/produto/10422/multilit-tubo-pvc-esg-75'
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until="networkidle")

        # pega o HTML renderizado
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        # tenta pegar o JSON do __NEXT_DATA__
        script = soup.find("script", id="__NEXT_DATA__")
        if script:
            data = json.loads(script.string)
            produto = data.get("props", {}).get("pageProps", {}).get("produto", {})
            url_img = data.get("props", {}).get("pageProps", {}).get("seo", {}).get("imageUrl", 'Não disponivel')
            print(url_img)
            print("Marca:", next((p.get("desc") for p in produto.get("dimensoes", []) if p.get("label") == "MARCA"), "não disponível"))
            print("Peso:", produto.get("pesoBruto", "não disponível"))
            print("Código de Barras:", produto.get("codBarra", "não disponível"))
        else:
            print("❌ Nenhum <script id=__NEXT_DATA__> encontrado")

        await browser.close()

# 👉 Em Jupyter/IPython:


await run()


In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process

produtos = pd.read_excel('produtos.xlsx')

corantes = produtos.query('Descrição == ')

# # Comparar descrições e achar semelhantes
# produtos['Grupo'] = None
# for i, desc in enumerate(produtos['Descrição']):
#     match = process.extractOne(desc, produtos['Descrição'], scorer=fuzz.token_sort_ratio)
#     if match[1] > 85 and match[0] != desc:
#         produtos.at[i, 'Grupo'] = match[0]



produtos



In [ ]:
import pandas as pd
df = pd.read_excel('produtos.xlsx')

from rapidfuzz import process, fuzz

# Criar uma cópia do dataframe para trabalhar
df_fuzzy = df.copy()

# Criar um dicionário para armazenar grupos de produtos semelhantes
grupos = {}
grupo_id = 1

# Threshold de similaridade (ajustável)
threshold = 75

# Lista para armazenar o grupo de cada item
grupo_coluna = []

for i, desc in enumerate(df_fuzzy['Descrição']):
    # Verificar se já foi agrupado
    if any(desc in v for v in grupos.values()):
        for gid, itens in grupos.items():
            if desc in itens:
                grupo_coluna.append(gid)
                break
        continue
    
    # Criar novo grupo
    grupos[grupo_id] = [desc]
    grupo_coluna.append(grupo_id)
    
    # Procurar similares
    matches = process.extract(desc, df_fuzzy['Descrição'], scorer=fuzz.token_sort_ratio, limit=None)
    for match_desc, score, idx in matches:
        if score >= threshold and match_desc != desc:
            grupos[grupo_id].append(match_desc)
    
    grupo_id += 1

# Adicionar coluna com id do grupo
df_fuzzy['Grupo'] = grupo_coluna

print(df_fuzzy.head(50))
df_fuzzy.to_excel('Fuzzy.xlsx', index=False)




In [6]:
df_fuzzy

,Codigo Produto,Descrição,Valor_unitário,Código de Barras,Sku,Fornecedor,Data Emissão,Marca,Peso,Url Imagem,Grupo
0,130416.0,SUP LAJE PE FIXO 30CM AM,3.540,7898759180523,7898759180523,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2025-10-03 05:27:00,INJEREST,0.13500000,https://cdn.construja.com.br/produtos/130416/1...,1
1,112142.0,CJ STYLUS 4X2 BR (1S) 2017,3.390,7898322640386,7898322640386,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2025-10-03 05:27:00,ILUMI,0.03700000,https://cdn.construja.com.br/produtos/112142/1...,2
2,199848.0,MARTELO BOLA POLIDO 300G,21.846,7899612781512,7899612781512,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2025-10-03 05:27:00,SPARTA,0.42400000,https://cdn.construja.com.br/produtos/199848/1...,3
3,132367.0,TORN BOIA METAL 3/4 TOP,22.397,7898945760096,TORN BOIA METAL 3/4 TOP,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2025-10-03 05:27:00,PERFECT,0.19500000,https://cdn.construja.com.br/produtos/132367/1...,4
4,132366.0,TORN BOIA METAL 3/4 FRIGIDEIRA,21.960,7898945760133,TORN BOIA METAL 3/4 FRIGIDEIRA,CONSTRUJA DISTR. DE MATERIAIS P/ CONSTRU,2025-10-03 05:27:00,PERFECT,0.19800000,https://cdn.construja.com.br/produtos/132366/1...,5
...,...,...,...,...,...,...,...,...,...,...,...
4425,319503.0,"OTTO B.VEDAPREN PAREDE BR GL 3,6KG GALAO",56.225,7897321120219,7897321120219,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2019-12-17 07:41:05,NaN,NaN,NaN,1939
4426,249033.0,CHAVE PHIL.FOXLUX 1/4 X 6,7.855,7898105776318,7898105776318,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2019-12-17 07:41:05,NaN,NaN,NaN,1190
4427,70270.0,TRINCHA CONDOR 700 C 1 PRETA,1.331,7891055169803,7891055169803,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2019-12-17 07:41:05,NaN,NaN,NaN,1456
4428,70262.0,TRINCHA CONDOR 700 B 3/4 PRETA,1.080,7891055169704,7891055169704,MJR CUNHA DISTRIBUIDORA DE MAT.CONSTR LTDA,2019-12-17 07:41:05,NaN,NaN,NaN,1754


In [ ]:
"Categoria" = "101197"
"Nome do Produto" = df_fuzzy['Descrição']
"Descrição do Produto" = df_fuzzy['Descrição'] + df_fuzzy['Marca']
"SKU PRINCIPAL" = df_fuzzy['Sku']
"Número de Integração de Variação" = df_fuzzy['Grupo']
"Nome da Variação 1" = "Variação"
"Opção para Variação 1" = df_fuzzy['Descrição']
"Imagem por Variação" = df_fuzzy['Url Imagem']
"Preço" = df_fuzzy['Preço']
"Estoque" = 0
"SKU da Variação" = df_fuzzy['Sku']
"GTIN (EAN)" = df_fuzzy['Código de Barras']
"Imagem de capa" =  df_fuzzy['Url Imagem']
"Imagem do produto 1" = df_fuzzy['Url Imagem']
"Peso" = df_fuzzy['Peso']
"Comprimento" = 10
"Largura" = 10
"Altura" = 10
"Shopee Xpress CPF"
"Retirada pelo Comprador"
"Marca"


Mercado LIVRE:50% do valor do produto (para produtos até R$ 12,50), R$ 6,25 (entre R$ 12,50 e R$ 29), R$ 6,50 (entre R$ 29 e R$ 50) e R$ 6,75 (entre R$ 50 e R$ 79)

Comissão de Categoria Construção 12%


